# SynthSeg Organs & FCD Lesion Segmentation

## Subject & Sequence Info

In [44]:
subject_number = 1              # Subject index (1 → 150)
sequence_type = "T1"          # Image sequence: "T1" or "FLAIR"

## Required Libraries

In [12]:
import os
import shutil

import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.image import resample_img

## Directory Utilities

In [13]:
def delete_directory(dir_path, self_delete=False):
    """
    Deletes all contents of a directory. Optionally deletes the directory itself.

    Args:
        dir_path (str): Path to the directory to clear.
        self_delete (bool): If True, deletes the directory itself after clearing contents.
    """
    if not os.path.exists(dir_path):
        print(f"Path does not exist: {dir_path}")
        return

    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
        except Exception as e:
            print(f"Failed to delete {item_path}: {e}")

    if self_delete:
        try:
            shutil.rmtree(dir_path)
        except Exception as e:
            print(f"Failed to delete directory itself: {e}")


def clear_kaggle_working():
    """Clears the '/kaggle/working' directory."""
    kaggle_working_path = "/kaggle/working"
    delete_directory(kaggle_working_path)


def print_folder_structure(start_path, indent=""):
    """
    Recursively prints the directory structure in a tree-like format.

    Args:
        start_path (str): Directory path to start printing from.
        indent (str): Indentation for nested levels.
    """
    if not os.path.exists(start_path):
        print(f"Error: Path '{start_path}' does not exist.")
        return

    if not os.path.isdir(start_path):
        print(f"Error: Path '{start_path}' is not a directory.")
        return

    entries = sorted(os.listdir(start_path))  # Sort entries for consistent output

    for i, entry in enumerate(entries):
        entry_path = os.path.join(start_path, entry)
        is_last = i == len(entries) - 1

        prefix = "└── " if is_last else "├── "
        print(f"{indent}{prefix}{entry}")

        if os.path.isdir(entry_path):
            next_indent = indent + ("    " if is_last else "│   ")
            print_folder_structure(entry_path, next_indent)


In [19]:
delete_directory("/kaggle/working/output")

## Setup Kaggle Working Directory

In [17]:
%cd /kaggle/working/

# Clone the SynthSeg repository
!git clone https://github.com/GabrieleLozupone/SynthSeg-TF2.15-Dataset.git synthseg_modern


!mkdir -p output

/kaggle/working
Cloning into 'synthseg_modern'...
remote: Enumerating objects: 2374, done.
remote: Counting objects: 100% (692/692), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 2374 (delta 617), reused 531 (delta 531), pack-reused 1682 (from 3)
Receiving objects: 100% (2374/2374), 120.58 MiB | 43.97 MiB/s, done.
Resolving deltas: 100% (1533/1533), done.


## Install Repository Dependencies

In [5]:
# Navigate to the repository root
%cd /kaggle/working/synthseg_modern

# Install Python dependencies from requirements.txt
!pip install -r requirements.txt

print("\n✔️ All dependencies installed successfully.")

/kaggle/working/SynthSeg-Modern
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 MB 68.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 123.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 151.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 161.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.8/845.8 kB 198.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 MB 46.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 117.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 136.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 MB 107.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 MB 92.7 MB/s eta 0:00:0000:0100:01
 

## SynthSeg Prediction for one subject

### Organize Output Directory

In [45]:
# ==============================
# --- Configuration Section ---
# ==============================
synthseg_home = "/kaggle/working/synthseg_modern"
dataset_root = "/kaggle/input/fcd-subject-centric/FCD-Subject-Centric Dataset"
base_output_dir = "/kaggle/working/output"

use_gpu = True  # Flag to enable GPU if available

# ==============================
# --- Path Setup & Validation ---
# ==============================
sequence_type = sequence_type.upper()
if sequence_type not in ["T1", "FLAIR"]:
    raise ValueError(f"Invalid sequence type '{sequence_type}'. Must be 'T1' or 'FLAIR'.")

image_path = os.path.join(
    dataset_root, f"sub-{subject_number:05d}", f"{sequence_type.lower()}.nii"
)
if not os.path.isfile(image_path):
    raise FileNotFoundError(f"Input image not found: {image_path}")

# --- Output Directory Structure ---
subject_output_dir = os.path.join(base_output_dir, f"sub-{subject_number:05d}", sequence_type)
synthseg_dir = os.path.join(subject_output_dir, "synthseg")
lesions_dir = os.path.join(subject_output_dir, "lesions")
fusion_dir = os.path.join(subject_output_dir, "fusion")

for dir_path in [synthseg_dir, lesions_dir, fusion_dir]:
    os.makedirs(dir_path, exist_ok=True)

### Model Inference

In [46]:
from SynthSeg.predict import predict

# ==============================
# --- Model & Data Paths ---
# ==============================
model_path = os.path.join(synthseg_home, "models", "synthseg_1.0.h5")
data_dir = os.path.join(synthseg_home, "data", "labels_classes_priors")

labels_segmentation_path = os.path.join(data_dir, "synthseg_segmentation_labels.npy")
names_segmentation_path = os.path.join(data_dir, "synthseg_segmentation_names.npy")
topology_classes_path = os.path.join(data_dir, "synthseg_topological_classes.npy")

# ==============================
# --- SynthSeg Output Paths ---
# ==============================
segmentation_path = os.path.join(synthseg_dir, "segmentation.nii.gz")
posteriors_path = os.path.join(synthseg_dir, "posteriors.nii.gz")
resampled_path = os.path.join(synthseg_dir, "resampled.nii.gz")
volumes_path = os.path.join(synthseg_dir, "volumes.csv")

# ==============================
# --- Prediction Parameters ---
# ==============================
cropping_size = 192
target_resolution = 1.0

flip_input = True
n_neutral_labels = 18
smoothing_sigma = 0.5
keep_largest_component = True

n_levels = 5
conv_per_level = 2
conv_kernel_size = 3
unet_features = 24
activation_function = "elu"
feature_multiplier = 2

gt_folder = None
evaluation_labels = None
list_incorrect_labels = None
list_correct_labels = None
compute_distances = False

# ==============================
# --- Device Setup ---
# ==============================
if use_gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    print("\nUsing GPU (CUDA_VISIBLE_DEVICES=0)")
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    print("\nUsing CPU (CUDA_VISIBLE_DEVICES=-1)")

# ==============================
# --- Resample Input Image ---
# ==============================
print(f"\nPreparing resampled image at {target_resolution} mm -> {resampled_path}")
try:
    from nibabel.processing import resample_to_output

    img = nib.load(image_path)
    resampled_img = resample_to_output(img, voxel_sizes=(target_resolution,) * 3)
    nib.save(resampled_img, resampled_path)
    print(f"✔️ Saved resampled image: {resampled_path}")

except Exception as e:
    print(f"Warning: resampling failed ({e}). Copying original image instead.")
    try:
        shutil.copy(image_path, resampled_path)
        print(f"✔️ Copied original image to {resampled_path}")
    except Exception as e2:
        raise RuntimeError(f"Failed to create resampled file: {e2}")

# ==============================
# --- Run SynthSeg Prediction ---
# ==============================
print("\n--- Starting SynthSeg Prediction ---")
print(f"Subject: sub-{subject_number:05d}")
print(f"Sequence: {sequence_type}")
print(f"Input Image: {resampled_path}")
print(f"Model: {model_path}")
print(f"Output Directory: {synthseg_dir}\n")

predict(
    path_images=resampled_path,
    path_segmentations=segmentation_path,
    path_model=model_path,
    labels_segmentation=labels_segmentation_path,
    names_segmentation=names_segmentation_path,
    path_posteriors=posteriors_path,
    path_resampled=resampled_path,
    path_volumes=volumes_path,
    cropping=cropping_size,
    target_res=target_resolution,
    flip=flip_input,
    topology_classes=topology_classes_path,
    sigma_smoothing=smoothing_sigma,
    keep_biggest_component=keep_largest_component,
    n_levels=n_levels,
    nb_conv_per_level=conv_per_level,
    conv_size=conv_kernel_size,
    unet_feat_count=unet_features,
    feat_multiplier=feature_multiplier,
    activation=activation_function,
    gt_folder=gt_folder,
    evaluation_labels=evaluation_labels,
    list_incorrect_labels=list_incorrect_labels,
    list_correct_labels=list_correct_labels,
    compute_distances=compute_distances,
    recompute=True,
    verbose=True,
)

print("\n✔️ Prediction complete! Output files:")
for path in [segmentation_path, posteriors_path, resampled_path, volumes_path]:
    print(f" - {path}")


Using GPU (CUDA_VISIBLE_DEVICES=0)

Preparing resampled image at 1.0 mm -> /kaggle/working/output/sub-00001/T1/synthseg/resampled.nii.gz
✔️ Saved resampled image: /kaggle/working/output/sub-00001/T1/synthseg/resampled.nii.gz

--- Starting SynthSeg Prediction ---
Subject: sub-00001
Sequence: T1
Input Image: /kaggle/working/output/sub-00001/T1/synthseg/resampled.nii.gz
Model: /kaggle/working/synthseg_modern/models/synthseg_1.0.h5
Output Directory: /kaggle/working/output/sub-00001/T1/synthseg

predicting 1/1
1/1 [==============================] - 1s 683ms/step

✔️ Prediction complete! Output files:
 - /kaggle/working/output/sub-00001/T1/synthseg/segmentation.nii.gz
 - /kaggle/working/output/sub-00001/T1/synthseg/posteriors.nii.gz
 - /kaggle/working/output/sub-00001/T1/synthseg/resampled.nii.gz
 - /kaggle/working/output/sub-00001/T1/synthseg/volumes.csv


## Lesion Mask Preparation

In [47]:
# Define lesion output directory
lesion_output_dir = os.path.join(subject_output_dir, "lesions")
os.makedirs(lesion_output_dir, exist_ok=True)

# --- Input & Output Paths ---
lesion_path = os.path.join(dataset_root, f"sub-{subject_number:05d}", "roi.nii")
roi_original_path = os.path.join(lesion_output_dir, "roi_original.nii.gz")
resampled_lesion_path = os.path.join(lesion_output_dir, "roi_resampled.nii.gz")

# --- Save original lesion mask as .nii.gz ---
if os.path.isfile(lesion_path):
    roi_img = nib.load(lesion_path)          # Load original ROI
    nib.save(roi_img, roi_original_path)     # Save as compressed .nii.gz
else:
    raise FileNotFoundError(
        f"Lesion mask not found for subject sub-{subject_number:05d}.\n"
        f"Expected at: {lesion_path}\n"
        "This subject may be a Healthy Control (no ROI file present)."
    )

# --- Validate SynthSeg resampled reference ---
if not os.path.isfile(resampled_path):
    raise FileNotFoundError(
        f"Reference image for resampling not found at:\n{resampled_path}\n"
        "Ensure SynthSeg finished successfully before running lesion resampling."
    )

# --- Load target space from SynthSeg output ---
target_img = nib.load(resampled_path)
target_shape = target_img.shape
target_affine = target_img.affine

# --- Resample lesion mask ---
print("\n--- Starting Lesion Mask Resampling ---")
roi_img = nib.load(roi_original_path)

print(f"Original lesion mask shape: {roi_img.shape}")
print(f"Target (SynthSeg) shape: {target_shape}")

try:
    resampled_roi_img = resample_img(
        img=roi_img,
        target_affine=target_affine,
        target_shape=target_shape,
        interpolation="nearest"  # Preserve binary values
    )
    nib.save(resampled_roi_img, resampled_lesion_path)

    print(f"\n✔️ Lesion mask resampling complete.")
    print(f"Resampled lesion saved at: {resampled_lesion_path}")
    print(f"Resampled lesion shape: {resampled_roi_img.shape}")

except Exception as e:
    print(f"\nResampling failed: {e}")
    print("Falling back to copying the original lesion mask (unresampled).")
    fallback_path = os.path.join(lesion_output_dir, "roi_original_copy.nii.gz")
    try:
        shutil.copy(roi_original_path, fallback_path)
        print(f"✔️ Copied original ROI mask to: {fallback_path}")
    except Exception as e2:
        raise RuntimeError(f"Failed to copy fallback ROI file: {e2}")


--- Starting Lesion Mask Resampling ---
Original lesion mask shape: (160, 256, 256)
Target (SynthSeg) shape: (193, 289, 293)

✔️ Lesion mask resampling complete.
Resampled lesion saved at: /kaggle/working/output/sub-00001/T1/lesions/roi_resampled.nii.gz
Resampled lesion shape: (193, 289, 293)


## Anatomical Brain & FCD Lesion Fusion

In [48]:
lesion_class_id = 99  # Label ID for FCD lesion

# --- Paths ---
fusion_dir = os.path.join(subject_output_dir, "fusion")
fusion_path = os.path.join(fusion_dir, "final_segmentation_mask.nii.gz")
os.makedirs(fusion_dir, exist_ok=True)

# --- SynthSeg segmentation path ---
synthseg_segmentation_path = os.path.join(subject_output_dir, "synthseg", "segmentation.nii.gz")

# --- 1. Load SynthSeg anatomical segmentation ---
if not os.path.isfile(synthseg_segmentation_path):
    raise FileNotFoundError(
        f"SynthSeg segmentation not found at:\n{synthseg_segmentation_path}"
    )

synthseg_img = nib.load(synthseg_segmentation_path)
fused_data = synthseg_img.get_fdata().astype(np.int16)
affine_synthseg = synthseg_img.affine

# --- 2. Load resampled FCD lesion ROI ---
if not os.path.isfile(resampled_lesion_path):
    raise FileNotFoundError(
        f"Resampled lesion mask not found at:\n{resampled_lesion_path}"
    )

resampled_lesion_img = nib.load(resampled_lesion_path)
resampled_roi_data = resampled_lesion_img.get_fdata().astype(np.int8)

# --- 3. Sanity checks ---
if fused_data.shape != resampled_roi_data.shape:
    raise ValueError(
        f"Shape mismatch: SynthSeg {fused_data.shape} vs lesion {resampled_roi_data.shape}"
    )

if not np.allclose(affine_synthseg, resampled_lesion_img.affine, atol=1e-3):
    print("Warning: Affine mismatch detected. Fusion may be spatially misaligned.")

# --- 4. Fuse lesion into anatomical segmentation ---
lesion_voxel_indices = np.where(resampled_roi_data == 1)

if lesion_voxel_indices[0].size == 0:
    print("No lesion voxels detected. Fusion will proceed unchanged.")
else:
    print(f"Adding {lesion_voxel_indices[0].size:,} lesion voxels (label={lesion_class_id}).")
    fused_data[lesion_voxel_indices] = lesion_class_id

# --- 5. Save fused segmentation ---
try:
    # Remove existing file/folder if present
    if os.path.isdir(fusion_path):
        shutil.rmtree(fusion_path)
    elif os.path.exists(fusion_path):
        os.remove(fusion_path)

    fused_img = nib.Nifti1Image(fused_data, affine_synthseg)
    nib.save(fused_img, fusion_path)

    print(f"\n✔️ Fusion complete!")
    print(f"Final fused segmentation saved at: {fusion_path}")
    print(f"Image shape: {fused_data.shape}")

except Exception as e:
    raise RuntimeError(f"Failed to save fused segmentation: {e}")


Adding 2,586 lesion voxels (label=99).

✔️ Fusion complete!
Final fused segmentation saved at: /kaggle/working/output/sub-00001/T1/fusion/final_segmentation_mask.nii.gz
Image shape: (193, 289, 293)
